# Sensitivity analysis

Sensitivity analysis (SA) studies the uncertainty in the output of a model. It attempts to measure this uncertainty in the output by making small (known) changes in the inputs, in the process identifying which parameter/set of parameters has the greatest influence. This increases understanding of the relationship between input and output, which in turn can help us find errors in the model. Another purpose of SA is model simplification; model inputs that have little to no effect on the output can be fixed to specific values.

In this notebook we aim to familiarise you with OFAT and Sobol SA. We will use Mesa's BatchRunner throughout the notebook and Sobol SA will be done through use of [SALib](https://github.com/SALib/SALib).

First, we install the package and test if we can import it.

In [1]:
from IPython.display import clear_output
!pip install SALib
import SALib
clear_output()
print("Everything A-Okay!")

Everything A-Okay!


Now we'll import everything required for this notebook...

In [2]:
%matplotlib inline
import sys
sys.path.append('C:/Users/User/PycharmProjects/MAS_Mesa_maze/Model/Model')

from SALib.sample import saltelli
from .model import WalkerModel
from .efficient_agents import RedWalker, CyanWalker
from mesa.batchrunner import BatchRunner
from SALib.analyze import sobol
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations

ModuleNotFoundError: No module named '__main__.model'; '__main__' is not a package

# The model

SA is only usefull if we can apply it to something. Instead of having you make another model, we will use a predator-prey model that you are familiar with; the "wolf-sheep predation model". Specifically, [the wolf-sheep model provided in the examples provided by Mesa](https://github.com/projectmesa/mesa/tree/master/examples/wolf_sheep). You might recognise parts of it that were used as inspiration for the first notebook.

From the description:

''A simple ecological model, consisting of three agent types: wolves, sheep, and grass. The wolves and the sheep wander around the grid at random. Wolves and sheep both expend energy moving around, and replenish it by eating. Sheep eat grass, and wolves eat sheep if they end up on the same grid cell.

If wolves and sheep have enough energy, they reproduce, creating a new wolf or sheep (in this simplified model, only one parent is needed for reproduction). The grass on each cell regrows at a constant rate. If any wolves and sheep run out of energy, they die.''

The model can be found in the folder 'wolf_sheep/'. If you are interested, you can read through the code. You can even use:

`mesa runserver`

...to view the model in action.

# OFAT SA
One Factor at A Time (OFAT or OFaaT) SA is a method of testing the output variance of inputs one at a time. 

We will show you how to change only one variable at the time and how to plot the results. The model we use has default parameters for each of the variables, which makes this fairly easy.

In [ ]:
# We define our variables and bounds
problem = {
    'num_vars': 3,
    'names': ['sheep_reproduce', 'wolf_reproduce', 'wolf_gain_from_food'],
    'bounds': [[0.01, 0.1], [0.01, 0.1], [10, 40]]
}

# Set the repetitions, the amount of steps, and the amount of distinct values per variable
replicates = 10
max_steps = 100
distinct_samples = 20 

# Set the outputs
model_reporters = {"Wolves": lambda m: m.schedule.get_breed_count(Wolf),
             "Sheep": lambda m: m.schedule.get_breed_count(Sheep)}

data = {}

for i, var in enumerate(problem['names']):
    # Get the bounds for this variable and get <distinct_samples> samples within this space (uniform)
    samples = np.linspace(*problem['bounds'][i], num=distinct_samples)
    
    # Keep in mind that wolf_gain_from_food should be integers. You will have to change
    # your code to acommidate for this or sample in such a way that you only get integers.
    if var == 'wolf_gain_from_food':
        samples = np.linspace(*problem['bounds'][i], num=distinct_samples, dtype=int)
    
    batch = BatchRunner(WolfSheep, 
                        max_steps=max_steps,
                        iterations=replicates,
                        variable_parameters={var: samples},
                        model_reporters=model_reporters,
                        display_progress=True)
    
    batch.run_all()
    
    data[var] = batch.get_model_vars_dataframe()
    

Now that we have the data, we can plot it! First show the influence of the variable on sheep, then on wolves.

In [ ]:
def plot_param_var_conf(ax, df, var, param, i):
    """
    Helper function for plot_all_vars. Plots the individual parameter vs
    variables passed.

    Args:
        ax: the axis to plot to
        df: dataframe that holds the data to be plotted
        var: variables to be taken from the dataframe
        param: which output variable to plot
    """
    x = df.groupby(var).mean().reset_index()[var]
    y = df.groupby(var).mean()[param]

    replicates = df.groupby(var)[param].count()
    err = (1.96 * df.groupby(var)[param].std()) / np.sqrt(replicates)

    ax.plot(x, y, c='k')
    ax.fill_between(x, y - err, y + err)

    ax.set_xlabel(var)
    ax.set_ylabel(param)

def plot_all_vars(df, param):
    """
    Plots the parameters passed vs each of the output variables.

    Args:
        df: dataframe that holds all data
        param: the parameter to be plotted
    """

    f, axs = plt.subplots(3, figsize=(7, 10))
    
    for i, var in enumerate(problem['names']):
        plot_param_var_conf(axs[i], data[var], var, param, i)

for param in ('Sheep', 'Wolves'):
    plot_all_vars(data, param)
    plt.show()

# Sobol SA

Sobol Sensitivity Analysis ([Sobol 2001](http://www.sciencedirect.com/science/article/pii/S0378475400002706), [Saltelli 2002](http://www.sciencedirect.com/science/article/pii/S0010465502002801), [Saltelli et al. 2010](http://www.sciencedirect.com/science/article/pii/S0010465509003087)) is a global SA method that determines the contribution of each input parameter or a combination of parameters and their interaction to the overall output variance. OFAT, while it is easier to implement (and certainly is less thought-intensive), has a couple of downsides.

1. OFAT requires a large amount of runs to get accurate results
2. OFAT cannot estimate interactions of combinations of inputs
3. OFAT can miss optimal settings of factors

Sobol can find higher order interactions, but still requires a large amount of runs.

### Getting the data

Before we can start analysing the model, we will have to sample our data. There are multiple methods for sampling included in SALib, but since we're using Sobol, we will use Saltelli sampling for this. 

The following code shows how you could collect data for the "wolf-sheep" model.

In [ ]:
# Set the repetitions, the amount of steps, and the amount of distinct values per variable
replicates = 10
max_steps = 100
distinct_samples = 10

# We get all our samples here
param_values = saltelli.sample(problem, distinct_samples)

# READ NOTE BELOW CODE
batch = BatchRunner(WolfSheep, 
                    max_steps=max_steps,
                    variable_parameters={name:[] for name in problem['names']},
                    model_reporters=model_reporters)

count = 0
for i in range(replicates):
    for vals in param_values: 
        # Change parameters that should be integers
        vals = list(vals)
        vals[2] = int(vals[2])

        # Transform to dict with parameter names and their values
        variable_parameters = {}
        for name, val in zip(problem['names'], vals):
            variable_parameters[name] = val

        batch.run_iteration(variable_parameters, tuple(vals), count)
        count += 1

        clear_output()
        print(f'{count / (len(param_values) * (replicates)) * 100:.2f}% done')
    
data = batch.get_model_vars_dataframe()

Note that even though we use the BatchRunner provided by Mesa, we do not use its full capabilities. Normally, you would set all parameters properly (at line 25) and then run the batchrunner with `batch.run_all()`. However, the batchrunner will then proceed to run every possible combination of the variables you have passed it. We already have the combinations (samples) we need, because we got those from SALib.

Preferably, you would want to save the results as a csv so you can load them more easily when analysing your results. 

Printing the data shows (a part of) our results:

In [ ]:
print(data)

As you can see we have 800 distinct results. One sample from saltelli-sampling results in $2^3=8$ different combinations. (We have 3 variables, and we take 2 distinct values per sample per variable. For more details, take a look at [Saltelli et al. 2010](http://www.sciencedirect.com/science/article/pii/S0010465509003087)). Since we take 10 samples, we get 80 combinations. We repeat each of these combinations 10 times, resulting in 800 results.

We only consider 2 outputs in this example; the amount of sheep and wolves at the end of the simulation. Can you think of more outputs?

### Analyzing

Now we can use the `analyze()` method provided by SALib that performs Sobol analysis. 

In [ ]:
Si_sheep = sobol.analyze(problem, data['Sheep'].as_matrix(), print_to_console=True)
Si_wolves = sobol.analyze(problem, data['Wolves'].as_matrix(), print_to_console=True)

This is not very insightfull. Let's make a function that can plot this.

In [ ]:
def plot_index(s, params, i, title=''):
    """
    Creates a plot for Sobol sensitivity analysis that shows the contributions
    of each parameter to the global sensitivity.

    Args:
        s (dict): dictionary {'S#': dict, 'S#_conf': dict} of dicts that hold
            the values for a set of parameters
        params (list): the parameters taken from s
        i (str): string that indicates what order the sensitivity is.
        title (str): title for the plot
    """

    if i == '2':
        p = len(params)
        params = list(combinations(params, 2))
        indices = s['S' + i].reshape((p ** 2))
        indices = indices[~np.isnan(indices)]
        errors = s['S' + i + '_conf'].reshape((p ** 2))
        errors = errors[~np.isnan(errors)]
    else:
        indices = s['S' + i]
        errors = s['S' + i + '_conf']
        plt.figure()

    l = len(indices)

    plt.title(title)
    plt.ylim([-0.2, len(indices) - 1 + 0.2])
    plt.yticks(range(l), params)
    plt.errorbar(indices, range(l), xerr=errors, linestyle='None', marker='o')
    plt.axvline(0, c='k')

We'll first plot 1st, 2nd, and total-order sensitivity for the output variable 'Sheep', then for 'Wolves'. 

In [ ]:
for Si in (Si_sheep, Si_wolves):
    # First order
    plot_index(Si, problem['names'], '1', 'First order sensitivity')
    plt.show()

    # Second order
    plot_index(Si, problem['names'], '2', 'Second order sensitivity')
    plt.show()

    # Total order
    plot_index(Si, problem['names'], 'T', 'Total order sensitivity')
    plt.show()